In [1]:
import numpy as np
import sympy as sp
import itertools as it
import string
import pickle

In [2]:
def commutazione(monomial, originals, inputs):
    while any(l2<l1 and l1+n1+l2+n2 in monomial for l1 in originals for l2 in originals for n1 in inputs for n2 in inputs):
        for l1 in originals:
            for l2 in originals:
                if l2<l1:
                    for n1 in inputs:
                        for n2 in inputs:
                            monomial = monomial.replace(l1+n1+l2+n2,l2+n2+l1+n1)
    return monomial

In [3]:
#definizioni 
parties = ['A','B','C','D','E','F','G','H']
originals = ['A', 'B', 'C']
inputs = ['0']
operators = [''.join(k) for k in it.product(parties,inputs, repeat = 1)]
originals_operators = [''.join(k) for k in it.product(originals,inputs, repeat = 1)]
level = 4
n=2

In [4]:
originals_mon_set = [''.join(k) for k in it.combinations(originals_operators, 1)] + [''.join(k) for k in it.combinations(originals_operators, 2)] + [''.join(k) for k in it.combinations(originals_operators, 3)]

In [5]:
non_inj=[]
for x in range(len(parties)-len(originals)):
    i=1
    count3=len(originals)*i+x
    count2=len(originals)-1+x
    while count3<len(parties):
        non_inj.append(parties[x]+parties[count3])
        #print(parties[x],parties[count3])
        non_inj.append(''.join(parties[x:x+len(originals)]))
        i+=1
        count3=len(originals)*i+x
    non_inj.append(''.join(parties[x+1:x+len(originals)+1]))
    while count2<len(parties):
        non_inj.append(parties[x]+parties[count2])
        count2+=len(originals)
        
        
#        non_inj.append(parties[x+len(originals)*m]+parties[x+len(originals)*(m+1)-1])
#        non_inj.append(parties[x+len(originals)*m:x*+len(originals)*(m+1)])
inflation_keys=sorted(sorted(set(non_inj)), key=len)
inflation_keys

['AC',
 'AD',
 'AF',
 'AG',
 'BD',
 'BE',
 'BG',
 'BH',
 'CE',
 'CF',
 'CH',
 'DF',
 'DG',
 'EG',
 'EH',
 'ABC',
 'BCD',
 'CDE',
 'DEF',
 'EFG',
 'FGH']

In [6]:
#f'matrici_mes_constraints_cut_inflation_nparties_{len(parties)}_lev_{level}'
filename = f'matrici_mes_constraints_cut_inflation_nparties_{len(parties)}_lev_{level}'
with open(filename, 'rb') as f:
    mes_diz = pickle.load(f)
filename = f'matrici_non_mes_constraints_cut_inflation_nparties_{len(parties)}_lev_{level}'
with open(filename, 'rb') as f:
    non_mes_diz = pickle.load(f)
filename = f'matrici_indep_constraints_cut_inflation_nparties_{len(parties)}_lev_{level}'
with open(filename, 'rb') as f:
    indep_cons_diz=pickle.load(f)

In [7]:
non_mes_keys={}
mes_keys={}
for i, j in enumerate(list(non_mes_diz.keys())):
    non_mes_keys[j]=i
for i, j in enumerate(list(mes_diz.keys())):
    mes_keys[j]=i

In [8]:
mat_dim = len(mes_diz[''][0])
observable_monomials = {originals_mon_set[i] : np.zeros((mat_dim,mat_dim)) for i in range(len(originals_mon_set))}

In [9]:
originals_mon_set

['A0', 'B0', 'C0', 'A0B0', 'A0C0', 'B0C0', 'A0B0C0']

In [10]:
index_keys=list(mes_diz.keys())
for i in range(len(index_keys)):
    lista=''.join([index_keys[i][j:j+1] for j in range(0, len(index_keys[i]), n) ]) 
    if any(non_inj[i] in lista for i in range(len(non_inj))):
        print('okay', lista)
        non_mes_diz[index_keys[i]]=mes_diz[index_keys[i]]
        mes_diz.pop(index_keys[i])

okay ABC
okay BCD
okay CDE
okay DEF
okay EFG
okay FGH
okay ABCD
okay BCDE
okay CDEF
okay DEFG
okay EFGH
okay ABCDE
okay BCDEF
okay CDEFG
okay DEFGH
okay ABCDEF
okay BCDEFG
okay CDEFGH
okay ABCDEFG
okay BCDEFGH
okay ABCDEFGH


In [11]:
index_keys=list(mes_diz.keys())
for i in range(1,len(index_keys)):
    tmp = index_keys[i]
    for j in range(len(parties)-len(originals)):
        tmp=tmp.replace(parties[len(originals)+j], parties[j%len(originals)])
    tmp=commutazione(tmp, parties, inputs)
    if tmp not in originals_mon_set:
        print(tmp)
        non_mes_diz[index_keys[i]]=mes_diz[index_keys[i]]
        mes_diz.pop(index_keys[i])

In [12]:
index_keys=list(mes_diz.keys())
for i in range(1,len(index_keys)):
    lista = index_keys[i]
    tmp = index_keys[i]
    for j in range(len(parties)-len(originals)):
        tmp=tmp.replace(parties[len(originals)+j], parties[j%len(originals)])
        #print(tmp)
    tmp=commutazione(tmp, originals, inputs)
    print(tmp)
    observable_monomials[tmp]+=mes_diz[index_keys[i]]

A0
B0
C0
A0
B0
C0
A0
B0
A0B0
B0C0
A0C0
A0B0
B0C0
A0C0
A0B0


In [13]:
len(observable_monomials['A0'][0])

163

In [14]:
for i in list(observable_monomials.keys()):
    if np.all(observable_monomials[i]==0):
        observable_monomials.pop(i)

In [15]:
index_keys=list(indep_cons_diz.keys())
indep_monomials={}
for i in range(len(index_keys)):
    tmp = index_keys[i]
    lista=list(list(indep_cons_diz.keys())[i].split('*'))
    for j in range(len(lista)):
        if lista[j] not in list(non_mes_diz.keys())+list(observable_monomials.keys()):
            for k in range(len(parties)-len(originals)):
                lista[j]=lista[j].replace(parties[len(originals)+k], parties[k%len(originals)])
            lista[j]=commutazione(lista[j], originals, inputs)
    lista=sorted(lista)
    indep_key='*'.join([lista[j] for j in range(len(lista))])
    if indep_key not in list(indep_monomials.keys()):
        indep_monomials[indep_key]=indep_cons_diz[tmp]
    else:
        indep_monomials[indep_key]+=indep_cons_diz[tmp]

In [16]:
indep_monomials

{'A0*C0': array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 1.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 1., 0., 0.]]),
 'A0*A0': array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]),
 'A0*B0': array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]),
 'B0*B0': array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [

In [17]:
filename = f'final_mat_mes_constraints_cut_inflation_nparties_{len(parties)}_lev_{level}'
with open(filename, 'wb') as f:
    pickle.dump(observable_monomials, f)
filename = f'final_mat_non_mes_constraints_cut_inflation_nparties_{len(parties)}_lev_{level}'
with open(filename, 'wb') as f:
    pickle.dump(non_mes_diz, f)
filename = f'final_mat_indep_constraints_cut_inflation_nparties_{len(parties)}_lev_{level}'
with open(filename, 'wb') as f:
    pickle.dump(indep_monomials, f)

In [18]:
len(non_mes_diz.keys())

21

In [19]:
non_mes_diz.keys()

dict_keys(['A0B0C0', 'B0C0D0', 'C0D0E0', 'D0E0F0', 'E0F0G0', 'F0G0H0', 'A0B0C0D0', 'B0C0D0E0', 'C0D0E0F0', 'D0E0F0G0', 'E0F0G0H0', 'A0B0C0D0E0', 'B0C0D0E0F0', 'C0D0E0F0G0', 'D0E0F0G0H0', 'A0B0C0D0E0F0', 'B0C0D0E0F0G0', 'C0D0E0F0G0H0', 'A0B0C0D0E0F0G0', 'B0C0D0E0F0G0H0', 'A0B0C0D0E0F0G0H0'])

In [20]:
indep_monomials.keys()

dict_keys(['A0*C0', 'A0*A0', 'A0*B0', 'B0*B0', 'B0*C0', 'C0*C0', 'A0*A0B0', 'A0B0*B0', 'A0B0*C0', 'A0*A0C0', 'A0*B0*C0', 'A0*C0*C0', 'A0*A0*C0', 'A0*A0*A0', 'A0*A0*B0', 'A0*B0C0', 'A0*B0*B0', 'B0*B0C0', 'B0C0*C0', 'B0*B0*B0', 'A0C0*B0', 'B0*B0*C0', 'A0C0*C0', 'B0*C0*C0', 'A0B0C0*B0', 'A0B0C0*C0', 'A0*A0B0C0', 'A0B0*A0B0', 'A0*A0B0*C0', 'A0*A0*A0B0', 'A0*A0B0*B0', 'A0B0*B0C0', 'A0B0*B0*B0', 'A0B0*A0C0', 'A0B0*B0*C0', 'A0*C0D0E0', 'A0*A0C0*C0', 'A0*A0*A0C0', 'A0*A0C0*B0', 'A0*B0C0*C0', 'A0*A0*B0*C0', 'A0*B0*B0*C0', 'A0*B0*C0*C0', 'A0*D0E0F0', 'A0*E0F0G0', 'A0*B0*B0C0', 'A0*F0G0H0', 'B0C0D0*C0', 'A0*B0C0D0', 'B0*B0C0D0', 'B0C0*B0C0', 'B0*B0*B0C0', 'A0C0*B0C0', 'B0*B0C0*C0', 'B0*D0E0F0', 'B0*E0F0G0', 'B0*F0G0H0', 'B0*C0D0E0', 'A0C0*A0C0', 'A0C0*B0*C0', 'C0*E0F0G0', 'C0*F0G0H0', 'A0B0C0D0*C0', 'A0*A0B0C0D0', 'A0B0C0D0*B0', 'A0B0C0*B0C0', 'A0*A0B0C0*B0', 'A0B0C0*B0*B0', 'A0B0C0*A0C0', 'A0B0C0*B0*C0', 'A0B0*A0B0C0', 'A0B0*D0E0F0', 'A0*A0B0*A0B0', 'A0B0*A0B0*B0', 'A0*A0B0*A0C0', 'A0*A0B0*B0*C0